# 交通拥堵指数代码整理
2020年华为云算法比赛赛题三算法数据处理思路整理


In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
from datetime import timedelta

data_url = '../data/train_TTI.csv' #use train_TTI.csv
TTI_df = pd.read_csv(data_url)
# road_id = TTI_df['id_road'].unique()
road_id = [276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269,
 276737, 276738]

# 构建关于TTI的数据集，记录前一小时各个路段的TTI和速度，'tti_{rid}_{minutes}'表示特定分钟之后对应路段的TTI
tti_dataset = {'road_id': [],'time':[],'recent': [],'TTI_0': [], 'TTI_1': [], 'TTI_2': []}  # TTI_0, TTI_1, TTI_2作为要预测的y值
for minutes in range(0, 60, 10):
    for rid in road_id:
        tti_dataset['tti_{}_{}'.format(rid, minutes)] = []
        tti_dataset['speed_{}_{}'.format(rid, minutes)] = []

startdateStr = '2019-01-01 00:00:00'  # 从一月第一天早晨开始建立
enddateStr = '2019-03-31 22:59:00'  # 从三月最后一天结束
datetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
startdatetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
enddatetimeObj = datetime.datetime.strptime(enddateStr, "%Y-%m-%d %H:%M:%S")
while datetimeObj <= enddatetimeObj:
    temp = datetimeObj
    print(temp.strftime("%Y-%m-%d %H:%M:%S"))
    try:
        temp_dict = {}
        temp_str = temp.strftime("%Y-%m-%d %H:%M:%S")
        temp_dict['time'] = temp_str
        for rid in road_id:
            for minutes in range(0, 90, 10):
                temp_str = (temp + timedelta(minutes=minutes)).strftime("%Y-%m-%d %H:%M:%S")
                tti_temp = float(TTI_df[(TTI_df['id_road'] == rid) & (TTI_df['time'] == temp_str)]['TTI'])
                speed_temp = float(TTI_df[(TTI_df['id_road'] == rid) & (TTI_df['time'] == temp_str)]['speed'])
                temp_dict['tti_{}_{}'.format(rid, minutes)]=tti_temp
                temp_dict['speed_{}_{}'.format(rid, minutes)]=speed_temp
    except:
        delta = timedelta(minutes=10)
        datetimeObj = datetimeObj + delta
        continue
    for thisid in road_id:
        tti_dataset['road_id'].append(thisid) # 要预测的路段
        tti_dataset['time'].append(temp_dict['time']) # 构建特征的起点时间，假如要预测当日9:00，9:10，9:20的TTI，以8:00为时间起点
        tti_dataset['recent'].append(temp_dict['tti_{}_{}'.format(thisid, 50)]) # 预测路段最后一个TTI的观察值
        tti_dataset['TTI_0'].append(temp_dict['tti_{}_{}'.format(thisid, 60)]) # 'TTI_{}'为要预测的y值
        tti_dataset['TTI_1'].append(temp_dict['tti_{}_{}'.format(thisid, 70)])
        tti_dataset['TTI_2'].append(temp_dict['tti_{}_{}'.format(thisid, 80)])

        for rid in road_id:
            for minutes in range(0, 60, 10):
                tti_dataset['tti_{}_{}'.format(rid, minutes)].append(temp_dict['tti_{}_{}'.format(rid, minutes)])
                tti_dataset['speed_{}_{}'.format(rid, minutes)].append(temp_dict['speed_{}_{}'.format(rid, minutes)])
    delta = timedelta(minutes=10)
    datetimeObj = datetimeObj + delta
    
startdateStr = '2019-10-01 00:00:00'  
enddateStr = '2019-12-21 22:59:00'  
datetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
startdatetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
enddatetimeObj = datetime.datetime.strptime(enddateStr, "%Y-%m-%d %H:%M:%S")
while datetimeObj <= enddatetimeObj:
    temp = datetimeObj
    print(temp.strftime("%Y-%m-%d %H:%M:%S"))
    try:
        temp_dict = {}
        temp_str = temp.strftime("%Y-%m-%d %H:%M:%S")
        temp_dict['time'] = temp_str
        for rid in road_id:
            for minutes in range(0, 90, 10):
                temp_str = (temp + timedelta(minutes=minutes)).strftime("%Y-%m-%d %H:%M:%S")
                tti_temp = float(TTI_df[(TTI_df['id_road'] == rid) & (TTI_df['time'] == temp_str)]['TTI'])
                speed_temp = float(TTI_df[(TTI_df['id_road'] == rid) & (TTI_df['time'] == temp_str)]['speed'])
                temp_dict['tti_{}_{}'.format(rid, minutes)]=tti_temp
                temp_dict['speed_{}_{}'.format(rid, minutes)]=speed_temp
    except:
        delta = timedelta(minutes=10)
        datetimeObj = datetimeObj + delta
        continue
    for thisid in road_id:
        tti_dataset['road_id'].append(thisid)
        tti_dataset['time'].append(temp_dict['time'])
        tti_dataset['recent'].append(temp_dict['tti_{}_{}'.format(thisid, 50)])
        tti_dataset['TTI_0'].append(temp_dict['tti_{}_{}'.format(thisid, 60)])
        tti_dataset['TTI_1'].append(temp_dict['tti_{}_{}'.format(thisid, 70)])
        tti_dataset['TTI_2'].append(temp_dict['tti_{}_{}'.format(thisid, 80)])

        for rid in road_id:
            for minutes in range(0, 60, 10):
                tti_dataset['tti_{}_{}'.format(rid, minutes)].append(temp_dict['tti_{}_{}'.format(rid, minutes)])
                tti_dataset['speed_{}_{}'.format(rid, minutes)].append(temp_dict['speed_{}_{}'.format(rid, minutes)])
    delta = timedelta(minutes=10)
    datetimeObj = datetimeObj + delta
pd.DataFrame(tti_dataset).to_csv('../data/more_TTI_preprosess.csv', index=False)


## GPS轨迹数据预处理和特征构建
GPS特征处理时主要采用在地图上采取关键点的方式进行。

1. 首先将每个订单的数据分散成单个的数据点，包含订单信息，时间、经纬度、方向以及速度。
2. 选取关键点经纬度和方向进行采样，每隔固定时间对这些点采样营运车辆的速度和流量信息
3. 类似于构建TTI特征的方式，以某个时间为起点，加入接下来一个小时各采样点的流量和速度作为特征

In [ ]:
# GPS处理第一步
from datetime import datetime
import pandas as pd
import numpy as np
import csv
import sys

def traj2csv(path, output):
    '''
    input: 文件路径
    output：输出文件，展开后的[userIDs, orderIDs, lngs, lats, speeds, directions, timestamps], list
    '''
    write_file = open(output, 'w', encoding='utf-8')
    csv_write = csv.writer(write_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_write.writerow(['id','UserID','OrderID','LNG','LAT','SPEED','DIRECTION','TIMESTAMP'])
    counter=0
    with open(path) as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            userIDs, orderIDs = row[0], row[1]
            traj = row[-1][1:-1]
            points = traj.split(', ')
            for point in points:
                counter+=1
                gps5_records = point.split(' ')
                to_write = [counter,userIDs, orderIDs,float(gps5_records[0]),float(gps5_records[1]),float(gps5_records[2]),float(gps5_records[3]),datetime.fromtimestamp(int(gps5_records[4]))]
                print(to_write)
                csv_write.writerow(to_write)
    write_file.close()

    
csv.field_size_limit(sys.maxsize)
data_list = ['../data/traffic2/toPredict_train_GPS_stage2.csv','../data/201901_201903.csv','../data/201910_11.csv','../data/20191201_20191220.csv']
out_list = ['../data/traffic2/toPredict_gps_stage2.csv','../data/gps201901_201903.csv','../data/gps201910_201911.csv','../data/gps20191201_20191220.csv']
for i in range(len(data_list)):
    traj2csv(data_list[i], out_list[i])

'''
# 转换后的数据(一个记录是一个gps点的5个字段)
UserID	OrderID	LNG	LAT	SPEED	DIRECTION	TIMESTAMP
xxxxxx  xxxxxxx xxx xxx xxxxx   xxxxxxxxx   xxxxxxxxx
xxxxxx  xxxxxxx xxx xxx xxxxx   xxxxxxxxx   xxxxxxxxx

拥有的标签为ID，经纬度，速度，方向，时间 方向为以正北方向为轴进行顺时针旋转的方向 userID为用户编号，
OrderID为行程编号，同一个OrderID可以对应不止一个userID 需要处理的主要文件:'./traffic1/toPredict_gps.csv', 
'./gps201910_201911.csv', './gps201901_201903.csv', './20191201_20191220.csv', 数据完整，没有缺失值
'''

In [ ]:
# GPS处理第二步
# 选取几个关键点，然后根据方向统计相关信息：研究时间间隔内的流量、平均速度、最大最小速度
import numpy as np
import time
import datetime
from datetime import timedelta
import pandas as pd
import random

keypoints = [[114.01619, 22.59639],[114.02036, 22.58824],[114.03132, 22.59794],
             [114.01962, 22.60689],[114.03029, 22.60736],[114.02983, 22.61313],
             [114.03332, 22.60873],[114.02789, 22.60277],[114.02570, 22.60831],
             [114.02594, 22.61078],[114.03243, 22.59199],[114.02634, 22.61345],
             [114.033524, 22.605982],[114.01562, 22.59151],[114.02367, 22.58933],
            [114.028857, 22.604546],[114.02996, 22.61016]]
keydirections = [[[0,90],[180,270]],[[45,90],[225,270]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,90],[180,270]],[[0,90],[180,270]],
                [[0,90],[180,270]],[[90,270],[270,90]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,180],[180,360]],[[90,270],[270,90]],
                 [[90,180],[270,360]],[[0,90],[180,270]],[[45,90],[225,270]],
                 [[0,90],[180,270]],[[90,180],[270,360]]]
grid_width = [0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005]
grid_height = [0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005]
        
startdateStr = ['2020-01-01 07:00:00','2019-10-01 00:00:00','2019-01-01 00:00:00','2019-12-01 00:00:00']
enddateStr = ['2020-01-13 22:00:00','2019-11-30 23:59:00','2019-03-31 23:59:00','2019-12-21 23:59:00']
file_list = [r'../data/traffic2/toPredict_gps_stage2.csv', r'../data/gps201910_201911.csv', r'../data/gps201901_201903.csv', r'../data/gps20191201_20191220.csv']
timeinterval = 5 #时间间隔

for timeidx in range(len(startdateStr)):
    features = {'time':[], 'volume':[], 'speedmean':[]}
    for i in range(len(keypoints)): # 不同的观测点
        for j in range(len(keydirections[i])): # 每个观测点不同的方向
            features['point{}direction{}volume'.format(i, j)] = []
            features['point{}direction{}speedmean'.format(i, j)] = []
#         features['point{}direction{}speedvar'.format(i, j)] = []
    datetimeObj = datetime.datetime.strptime(startdateStr[timeidx], "%Y-%m-%d %H:%M:%S")
    enddatetimeObj = datetime.datetime.strptime(enddateStr[timeidx], "%Y-%m-%d %H:%M:%S")
    path = file_list[timeidx]
    n = sum(1 for line in open(path)) - 1 #number of records in file (excludes header)
    s = int(n/10) #desired sample size
    skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
    df = pd.read_csv(path, infer_datetime_format=True, parse_dates=['TIMESTAMP'], skiprows=skip)
    while datetimeObj <= enddatetimeObj:
        start = datetimeObj
#         start = temp.strftime("%Y-%m-%d %H:%M:%S")
        delta = timedelta(minutes=timeinterval)
        end = start + delta
#         end = temp.strftime("%Y-%m-%d %H:%M:%S")
        print(start)
        temp_df = df[(df.TIMESTAMP >= start) & (df.TIMESTAMP < end)]
        if len(temp_df)==0:
            datetimeObj = datetimeObj + delta
            continue
        features['time'].append(start)
        # 区域内当前整体情况
        features['volume'].append(len(temp_df['OrderID'].unique()))
        features['speedmean'].append(temp_df['SPEED'].mean())
        for i in range(len(keypoints)): # 不同的观测点
            # 先对区域内点进行筛选
            temp_df1 = temp_df[(temp_df.LNG<(keypoints[i][0] + grid_width[i])) & (temp_df.LNG>(keypoints[i][0] - grid_width[i]))]
            temp_df1 = temp_df1[(temp_df1.LAT<(keypoints[i][1] + grid_height[i])) & (temp_df1.LAT>(keypoints[i][1] - grid_height[i]))]
            for j in range(len(keydirections[i])):
                if keydirections[i][j][0]<keydirections[i][j][1]:
                    temp_df2 = temp_df1[(temp_df1.DIRECTION>=keydirections[i][j][0]) & (temp_df1.DIRECTION<=keydirections[i][j][1])]
                else:
                    temp_df2 = temp_df1[(temp_df1.DIRECTION>=keydirections[i][j][0]) | (temp_df1.DIRECTION<=keydirections[i][j][1])]
                volume = len(temp_df2['OrderID'].unique())
                features['point{}direction{}volume'.format(i, j)].append(volume)
                mean_speed = temp_df2['SPEED'].mean()
                features['point{}direction{}speedmean'.format(i, j)].append(mean_speed)
#                 var_speed = temp_df2['SPEED'].var()
#                 features['point{}direction{}speedvar'.format(i, j)].append(var_speed)
        datetimeObj = datetimeObj + delta
    pd.DataFrame(features).to_csv('./gps_statistics_endmonth{}.csv'.format(enddatetimeObj.month), index=False)

# 将1月到12月的数据连起来
df1 = pd.read_csv('./gps_statistics_endmonth3.csv')
# df1['volume'] = df1['volume'].apply(lambda t:len(t))
df2 = pd.read_csv('./gps_statistics_endmonth11.csv')
# df2['volume'] = df2['volume'].apply(lambda t:len(t))
df3 = pd.read_csv('./gps_statistics_endmonth12.csv')
# df3['volume'] = df3['volume'].apply(lambda t:len(t))

df = pd.concat([df1, df2, df3], ignore_index=True)
df.fillna(0,inplace = True)
df.to_csv('train_gps_detailed.csv', index=False)


In [ ]:
# 类似对TTI数据的处理，统计前一小时各采样点的速度流量信息
import numpy as np
import pandas as pd
import time
import datetime
from datetime import timedelta
data_url = './train_gps_detailed2.csv'
gps_df = pd.read_csv(data_url, infer_datetime_format=True, parse_dates=['time'])
keypoints = [[114.01619, 22.59639],[114.02036, 22.58824],[114.03132, 22.59794],
             [114.01962, 22.60689],[114.03029, 22.60736],[114.02983, 22.61313],
             [114.03332, 22.60873],[114.02789, 22.60277],[114.02570, 22.60831],
             [114.02594, 22.61078],[114.03243, 22.59199],[114.02634, 22.61345],
             [114.033524, 22.605982],[114.01562, 22.59151],[114.02367, 22.58933],
            [114.028857, 22.604546],[114.02996, 22.61016]]
keydirections = [[[0,90],[180,270]],[[45,90],[225,270]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,90],[180,270]],[[0,90],[180,270]],
                [[0,90],[180,270]],[[90,270],[270,90]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,180],[180,360]],[[90,270],[270,90]],
                 [[90,180],[270,360]],[[0,90],[180,270]],[[45,90],[225,270]],
                 [[0,90],[180,270]],[[90,180],[270,360]]]
grid_width = [0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005,0.0005,
             0.0005,0.0005]
grid_height = [0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005,0.0005,
               0.0005,0.0005]

timeinterval = 5
gps_dataset = {'time':[]}
n_keypoints = len(keypoints)
for i in range(n_keypoints): # 不同的观测点
    for j in range(len(keydirections[i])): # 每个观测点不同的方向
        for minutes in range(0, 60, timeinterval):
            gps_dataset['point{}direction{}volume{}'.format(i, j, minutes)] = []
            gps_dataset['point{}direction{}speedmean{}'.format(i, j, minutes)] = []
for minutes in range(0, 60, timeinterval):
    gps_dataset['volume{}'.format(minutes)] = []
    gps_dataset['speedmean{}'.format(minutes)] = []
#             gps_dataset['point{}direction{}speedvar{}'.format(i, j, minutes)] = []

startdateStr = '2019-01-01 00:00:00'  # 从一月第一天早晨开始建立
enddateStr = '2019-03-31 12:59:00'  # 从三月最后一天结束
datetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
startdatetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
enddatetimeObj = datetime.datetime.strptime(enddateStr, "%Y-%m-%d %H:%M:%S")

while datetimeObj <= enddatetimeObj:
    temp = datetimeObj
    temp_dict = {}
    temp_str = temp.strftime("%Y-%m-%d %H:%M:%S")
    gps_dataset['time'].append(temp_str)
    for minutes in range(0, 60, timeinterval):
        temp_df = gps_df[gps_df['time'] == temp + timedelta(minutes=minutes)]
        gps_dataset['volume{}'.format(minutes)].append(int(temp_df['volume']))
        gps_dataset['speedmean{}'.format(minutes)].append(float(temp_df['speedmean']))
        
        for i in range(n_keypoints):
            for j in range(len(keydirections[i])):
                if int(temp_df['point{}direction{}volume'.format(i, j)])==0:
                    volume_temp = 0
                    speedm_temp = 0
#                     speedv_temp = float(temp_df['point{}direction{}speedvar'.format(i, j)])
                else:
                    volume_temp = int(temp_df['point{}direction{}volume'.format(i, j)])
                    speedm_temp = float(temp_df['point{}direction{}speedmean'.format(i, j)])
#                     speedv_temp = 0

                gps_dataset['point{}direction{}volume{}'.format(i, j, minutes)].append(volume_temp)
                gps_dataset['point{}direction{}speedmean{}'.format(i, j, minutes)].append(speedm_temp)
#                 gps_dataset['point{}direction{}speedvar{}'.format(i, j, minutes)].append(speedv_temp)
    delta = timedelta(minutes=10)
    datetimeObj = datetimeObj + delta
    
startdateStr = '2019-10-01 00:00:00'  
enddateStr = '2019-12-20 22:59:00'
datetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
startdatetimeObj = datetime.datetime.strptime(startdateStr, "%Y-%m-%d %H:%M:%S")
enddatetimeObj = datetime.datetime.strptime(enddateStr, "%Y-%m-%d %H:%M:%S")
while datetimeObj <= enddatetimeObj:
    temp = datetimeObj
    temp_dict = {}
    temp_str = temp.strftime("%Y-%m-%d %H:%M:%S")
    gps_dataset['time'].append(temp_str)
    print(temp_str)
    for minutes in range(0, 60, timeinterval):
        temp_df = gps_df[gps_df['time'] == temp + timedelta(minutes=minutes)]
        try:
            gps_dataset['volume{}'.format(minutes)].append(int(temp_df['volume']))
            gps_dataset['speedmean{}'.format(minutes)].append(int(temp_df['speedmean']))
        except:
            gps_dataset['volume{}'.format(minutes)].append(0)
            gps_dataset['speedmean{}'.format(minutes)].append(0)
        for i in range(n_keypoints):
            for j in range(len(keydirections[i])):
                if int(temp_df['point{}direction{}volume'.format(i, j)])==0:
                    volume_temp = 0
                    speedm_temp = 0
#                     speedv_temp = float(temp_df['point{}direction{}speedvar'.format(i, j)])
                else:
                    volume_temp = int(temp_df['point{}direction{}volume'.format(i, j)])
                    speedm_temp = float(temp_df['point{}direction{}speedmean'.format(i, j)])

                gps_dataset['point{}direction{}volume{}'.format(i, j, minutes)].append(volume_temp)
                gps_dataset['point{}direction{}speedmean{}'.format(i, j, minutes)].append(speedm_temp)
#                 gps_dataset['point{}direction{}speedvar{}'.format(i, j, minutes)].append(speedv_temp)
    delta = timedelta(minutes=10)
    datetimeObj = datetimeObj + delta
    
    
pd.DataFrame(gps_dataset).to_csv('../data/gps_features_detailed.csv', index=False)


In [ ]:
# 构建预测集的GPS特征进行输入
import csv
import pandas as pd
import time
import datetime
from datetime import timedelta

road_id = [276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269,
 276737, 276738]
timeinterval = 5
to_predict_url = '../data/traffic2/toPredict_noLabel_stage2.csv'
TTI_url = '../data/traffic2/toPredict_train_TTI_stage2.csv'
gps_url = './gps_statistics_test_stage2.csv'
TTI_df = pd.read_csv(TTI_url, infer_datetime_format=True, parse_dates=['time'])
gps_df = pd.read_csv(gps_url, infer_datetime_format=True, parse_dates=['time'])
features = {}
# 因为21号已知，拿21号的预测数据进行validation

X_labels = ['time','road_id','recent']
for rid in road_id:
    for minutes in range(0,60,10):
        X_labels.append('tti_{}_{}'.format(rid,minutes))
        X_labels.append('speed_{}_{}'.format(rid,minutes))
n_keypoints = len(keypoints)
for i in range(n_keypoints):
    for j in range(len(keydirections[i])):# 每个观测点不同的方向
        for minutes in range(0, 60, timeinterval):
            X_labels.append('point{}direction{}volume{}'.format(i, j, minutes))
            X_labels.append('point{}direction{}speedmean{}'.format(i, j, minutes))
for minutes in range(0, 60, timeinterval):
    X_labels.append('volume{}'.format(minutes))
    X_labels.append('speedmean{}'.format(minutes))
for label in X_labels:
    features[label]=[]
to_predict_url = '../data/traffic1/toPredict_noLabel.csv'
with open(to_predict_url) as f:
    next(f)
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        if counter % 3 != 0:
            counter+=1
            continue
        counter+=1
        road, time = int(row[1]), row[2]
        print(road, time)
        datetimeObj = datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        features['road_id'].append(road)
        features['time'].append(datetimeObj + timedelta(minutes=-60))
        features['recent'].append(float(TTI_df[(TTI_df['id_road']==road) & (TTI_df['time']==datetimeObj+timedelta(minutes=(-10)))]['TTI']))
        for _ in road_id:
            for minutes in range(0, 60, 10):
                temp = datetimeObj+timedelta(minutes=(minutes-60))
                tti_temp = float(TTI_df[(TTI_df['id_road']==_) & (TTI_df['time']==temp)]['TTI'])
                speed_temp = float(TTI_df[(TTI_df['id_road']==_) & (TTI_df['time']==temp)]['speed'])
                features['tti_{}_{}'.format(_, minutes)].append(tti_temp)
                features['speed_{}_{}'.format(_, minutes)].append(speed_temp)

        for minutes in range(0, 60, timeinterval):
            temp_df = gps_df[gps_df['time'] == datetimeObj + timedelta(minutes=minutes-60)]
            features['volume{}'.format(minutes)].append(int(temp_df['volume']))
            features['speedmean{}'.format(minutes)].append(float(temp_df['speedmean']))

            for i in range(n_keypoints):
                for j in range(len(keydirections[i])): 
                    if int(temp_df['point{}direction{}volume'.format(i, j)])==0:
                        volume_temp = 0
                        speedm_temp = 0
#                     speedv_temp = float(temp_df['point{}direction{}speedvar'.format(i, j)])
                    else:
                        volume_temp = int(temp_df['point{}direction{}volume'.format(i, j)])
                        speedm_temp = float(temp_df['point{}direction{}speedmean'.format(i, j)])

                    features['point{}direction{}volume{}'.format(i, j, minutes)].append(volume_temp)
                    features['point{}direction{}speedmean{}'.format(i, j, minutes)].append(speedm_temp)
#                     features['point{}direction{}speedvar{}'.format(i, j, minutes)].append(speedv_temp)
        
to_predict_X = pd.DataFrame(features)
to_predict_X.to_csv('to_predict_x_gps_detailed3_stage2.csv', index=False)
